In [1]:
import theano
from theano_models import Merge
from theano_models.visualization import d3viz
import theano_models.probabilistic_models as pm
import theano_models.deterministic_models as dm
import theano_models.postmaps as post
from scipy.optimize import minimize
from schlichtanders.myfunctools import compose
from schlichtanders.mygenerators import replicate, takeN
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import IFrame

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
example_input = np.array([2,3,4])
mlp = dm.Mlp(input=example_input, hidden_sizes=[10,10], output_size=2,
             hidden_transfers=['rectifier', 'rectifier'], output_transfer='softmax')
mlp

Mlp { 'inputs': [TensorConstant{[ 2.  3.  4.]}],
  'outputs': AffineNonlinear3.outputs,
  'parameters': [weights, bias, weights2, bias2, weights3, bias3]}

In [3]:
noise = pm.GaussianNoise(mlp)
noise

GaussianNoise { 'inputs': [AffineNonlinear3.outputs],
  'outputs': GaussianNoise.outputs,
  'parameters': [],
  'parameters_positive': [var]}

In [4]:
noise.logP

GaussianNoise.logP { 'extra_inputs': [AffineNonlinear3.outputs],
  'inputs': [rv],
  'outputs': GaussianNoise.logP.outputs,
  'parameters': [],
  'parameters_positive': [var]}

In [5]:
noisy_mlp = Merge(noise, mlp)
noisy_mlp

Merge2 { 'inputs': [TensorConstant{[ 2.  3.  4.]}],
  'outputs': GaussianNoise.outputs,
  'parameters': [weights, bias, weights2, bias2, weights3, bias3],
  'parameters_positive': [var]}

In [6]:
optimizable = post.probabilistic_optimizer_postmap(noisy_mlp)
optimizable

{'loss': Elemwise{neg,no_inplace}.0,
 'loss_inputs': [rv, TensorConstant{[ 2.  3.  4.]}]}

In [7]:
loss = theano.function(optimizable['loss_inputs']+optimizable['parameters'], optimizable['loss'], mode="FAST_COMPILE", profile=True)

In [8]:
example_inputs = [expr.eval() for expr in optimizable['loss_inputs'][1:]+optimizable['parameters']]

In [9]:
loss([0.5, 0.5], *example_inputs)

array(1.8378770664093453)

In [10]:
d3viz(loss, "tmp/loss.html", match_by_names=True)
IFrame("tmp/loss.html", width=700, height=600)